In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import os
import cv2
from PIL import Image
import pandas as pd

2024-04-18 01:41:50.906267: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_path = "data"
# Read the CSV file containing filenames and labels
labels_df = pd.read_csv(os.path.join(data_path, "english.csv"))

# Define categories(labels)
CATEGORIES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z','0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# Define image size
IMG_SIZE = 200


In [3]:
# Step 3: Create Training Data

training = []

def createTrainingData():
    for index, row in labels_df.iterrows():
        img_path = os.path.join(data_path, "images", row['image'])
        # Check if the image file exists
        if not os.path.exists(img_path):
            print("Image file does not exist:", img_path)
            continue
        
        class_num = CATEGORIES.index(row['label'])
        img_array = cv2.imread(img_path)
        
        # Check if the image array is empty (i.e., image reading failed)
        if img_array is None:
            print("Error reading image:", img_path)
            continue
        
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training.append([new_array, class_num])

createTrainingData()


In [4]:
random.shuffle(training)

In [5]:
X =[]
y =[]
for features, label in training:
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [6]:
len(CATEGORIES)

62

In [7]:
X = X.astype('float32')
X /= 255
from keras.utils import to_categorical
Y = to_categorical(y, 62)
print(Y[100])
Y

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [9]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [10]:
y_train = to_categorical(y_train, num_classes=62)
y_test = to_categorical(y_test, num_classes=62)

In [11]:
from keras.layers import Dense
from keras.activations import softmax

# Example usage of Dense layer with softmax activation
dense_layer = Dense(units=62, activation=softmax)

In [12]:
import tensorflow as tf

batch_size = 16
nb_classes = 62  # Adjusted to match the number of classes in your data
nb_epochs = 5
img_rows, img_columns = 200, 200
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(nb_filters, (nb_conv, nb_conv), padding='same', activation=tf.nn.relu,
                           input_shape=(img_rows, img_columns, img_channel)),
    tf.keras.layers.MaxPooling2D((nb_pool, nb_pool), strides=2),
    tf.keras.layers.Conv2D(nb_filters, (nb_conv, nb_conv), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((nb_pool, nb_pool), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(nb_classes, activation=softmax)
])


/Users/mannbellani/Documents/NumberRecognition/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, validation_data=(X_test, y_test))


Epoch 1/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 76s 434ms/step - accuracy: 0.0477 - loss: 4.3097 - val_accuracy: 0.2273 - val_loss: 3.1477
Epoch 2/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 73s 429ms/step - accuracy: 0.4470 - loss: 2.1870 - val_accuracy: 0.3944 - val_loss: 2.3807
Epoch 3/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 73s 427ms/step - accuracy: 0.8408 - loss: 0.6237 - val_accuracy: 0.4164 - val_loss: 2.5178
Epoch 4/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 73s 427ms/step - accuracy: 0.9488 - loss: 0.1971 - val_accuracy: 0.4047 - val_loss: 3.0699
Epoch 5/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 76s 442ms/step - accuracy: 0.9771 - loss: 0.0793 - val_accuracy: 0.4164 - val_loss: 3.3621


In [14]:
model.save('NumRecognition2.h5')